In [47]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random
import string

class WordPuzzleGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Word Puzzle - Interleaved Words")
        self.master.configure(bg='lightblue')

        self.words_by_level = [
            ["TREE", "EARN", "NEAT", "RANT"],  # Level 1
            ["FIRE", "RICE", "NICE", "FINE"],  # Level 2
            ["MATH", "HEAT", "TEAM", "MEAT"],  # Level 3
        ]

        self.level = 0
        self.score = 0
        self.grid_size = 5
        self.word_grid = []
        self.found_words = []
        self.selected_cells = []
        self.hints_used = 0
        self.max_hints = 2

        self.create_widgets()
        self.load_level()

    def create_widgets(self):
        self.title_label = tk.Label(self.master, text="Word Puzzle Game", font=("Helvetica", 20, "bold"), bg='lightblue')
        self.title_label.pack(pady=10)

        top_frame = tk.Frame(self.master, bg='lightblue')
        top_frame.pack()

        self.level_label = tk.Label(top_frame, text=f"Level {self.level + 1}", font=("Helvetica", 14), bg='lightblue')
        self.level_label.pack(side=tk.LEFT, padx=20)

        self.score_label = tk.Label(top_frame, text=f"Score: {self.score}", font=("Helvetica", 14), bg='lightblue')
        self.score_label.pack(side=tk.RIGHT, padx=20)

        self.grid_frame = tk.Frame(self.master, bg='lightblue')
        self.grid_frame.pack(pady=10)

        self.buttons_frame = tk.Frame(self.master, bg='lightblue')
        self.buttons_frame.pack()

        self.submit_button = tk.Button(self.buttons_frame, text="Submit", command=self.submit_word)
        self.submit_button.grid(row=0, column=0, padx=10)

        self.hint_icon = Image.open("bulb.png").resize((20, 20), Image.ANTIALIAS)
        self.hint_img = ImageTk.PhotoImage(self.hint_icon)

        self.hint_button = tk.Button(self.buttons_frame, text="Hint ", image=self.hint_img, compound=tk.RIGHT, command=self.give_hint)
        self.hint_button.grid(row=0, column=1, padx=10)

    def load_level(self):
        if self.level >= len(self.words_by_level):
            messagebox.showinfo("Congratulations!", f"You completed all levels!\nFinal Score: {self.score}")
            self.master.quit()
            return

        self.level_label.config(text=f"Level {self.level + 1}")
        self.current_words = self.words_by_level[self.level]
        self.found_words = []
        self.hints_used = 0
        self.selected_cells = []

        self.create_grid()
        self.place_words()
        self.fill_remaining_with_random_letters()

    def create_grid(self):
        for widget in self.grid_frame.winfo_children():
            widget.destroy()
        self.word_grid = []
        for i in range(self.grid_size):
            row = []
            for j in range(self.grid_size):
                btn = tk.Button(self.grid_frame, text="", width=4, height=2, font=("Helvetica", 16),
                                command=lambda r=i, c=j: self.select_cell(r, c), bg="lightyellow")
                btn.grid(row=i, column=j, padx=2, pady=2)
                row.append(btn)
            self.word_grid.append(row)

    def place_words(self):
        positions = [(i, j) for i in range(self.grid_size) for j in range(self.grid_size)]
        random.shuffle(positions)

        directions = [(0, 1), (1, 0)]  # Horizontal or Vertical
        for word in self.current_words:
            placed = False
            random.shuffle(positions)
            for row, col in positions:
                random.shuffle(directions)
                for dr, dc in directions:
                    if self.can_place(word, row, col, dr, dc):
                        self.set_word(word, row, col, dr, dc)
                        placed = True
                        break
                if placed:
                    break

    def can_place(self, word, row, col, dr, dc):
        for i in range(len(word)):
            r, c = row + i * dr, col + i * dc
            if not (0 <= r < self.grid_size and 0 <= c < self.grid_size):
                return False
            cell_text = self.word_grid[r][c]['text']
            if cell_text != "" and cell_text != word[i]:
                return False
        return True

    def set_word(self, word, row, col, dr, dc):
        for i in range(len(word)):
            r, c = row + i * dr, col + i * dc
            self.word_grid[r][c].config(text=word[i], fg="black")

    def fill_remaining_with_random_letters(self):
        for row in self.word_grid:
            for btn in row:
                if btn['text'] == "":
                    btn.config(text=random.choice(string.ascii_uppercase), fg="gray")

    def select_cell(self, row, col):
        btn = self.word_grid[row][col]
        if (row, col) in self.selected_cells:
            self.selected_cells.remove((row, col))
            btn.config(bg="lightyellow")
        else:
            self.selected_cells.append((row, col))
            btn.config(bg="lightgreen")

    def submit_word(self):
        word = "".join([self.word_grid[r][c]['text'] for r, c in self.selected_cells])
        if word in self.current_words and word not in self.found_words:
            self.found_words.append(word)
            self.score += 10
            for r, c in self.selected_cells:
                self.word_grid[r][c].config(bg="lightblue")
        else:
            for r, c in self.selected_cells:
                self.word_grid[r][c].config(bg="red")
            self.master.after(500, lambda: [self.word_grid[r][c].config(bg="lightyellow") for r, c in self.selected_cells])

        self.selected_cells.clear()
        self.score_label.config(text=f"Score: {self.score}")

        if len(self.found_words) == len(self.current_words):
            self.level += 1
            self.load_level()

    def give_hint(self):
        if self.hints_used >= self.max_hints:
            messagebox.showinfo("Hint", "No more hints available!")
            return

        for word in self.current_words:
            if word not in self.found_words:
                for i in range(self.grid_size):
                    for j in range(self.grid_size):
                        if self.word_grid[i][j]['text'] == word[0]:
                            self.word_grid[i][j].config(bg="orange")
                            self.hints_used += 1
                            return


if __name__ == "__main__":
    root = tk.Tk()
    game = WordPuzzleGame(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Aafrren Mughal\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\Aafrren Mughal\AppData\Local\Temp\ipykernel_12348\884013677.py", line 165, in submit_word
    self.setup_level()
    ~~~~~~~~~~~~~~~~^^
  File "C:\Users\Aafrren Mughal\AppData\Local\Temp\ipykernel_12348\884013677.py", line 51, in setup_level
    self.generate_grid_with_words()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Aafrren Mughal\AppData\Local\Temp\ipykernel_12348\884013677.py", line 110, in generate_grid_with_words
    row = random.randint(0, GRID_SIZE - len(word))
  File "c:\Users\Aafrren Mughal\AppData\Local\Programs\Python\Python313\Lib\random.py", line 340, in randint
    return self.randrange(a, b+1)
           ~~~~~~~~~~~~~~^^^^^^^^
  File "c:\Users\Aafrren Mughal\AppData\Local\Programs\Python\Python313\Lib\ran

In [53]:
import tkinter as tk
from tkinter import messagebox
import random
import math

GRID_SIZE = 5
BUTTON_RADIUS = 30
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 700

class WordPuzzleGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Word Puzzle - Interleaved Words")
        self.root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")
        self.root.configure(bg='lightblue')

        self.canvas = tk.Canvas(root, width=WINDOW_WIDTH, height=WINDOW_HEIGHT, bg='lightblue', highlightthickness=0)
        self.canvas.pack()

        self.grid_buttons = []
        self.letter_buttons = []
        self.letter_coords = []
        self.selected_letters = []
        self.drag_line = None
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        self.score = 0
        self.level = 1
        self.hints_used = 0
        self.max_hints = 2
        self.found_words = []

        self.level_word_bank = {
            1: ["APPLE", "MANGO", "PEACH", "BERRY", "GRAPE"],
            2: ["LION", "TIGER", "ZEBRA", "BEAR", "HORSE"],
            3: ["RIVER", "OCEAN", "DESERT", "FOREST", "MOUNTAIN"]
        }

        self.create_widgets()
        self.setup_level()

    def setup_level(self):
        self.found_words = []
        self.hints_used = 0
        self.clear_grid()
        self.clear_canvas()
        self.clear_letters()
        self.level_words = self.pick_words_for_level()
        self.level_label.config(text=f"Level {self.level}")
        self.generate_grid_with_words()
        self.create_letter_buttons()

    def pick_words_for_level(self):
        return self.level_word_bank.get(self.level, random.sample(list(self.level_word_bank[1]), 5))

    def create_widgets(self):
        self.title_label = tk.Label(self.root, text="Word Puzzle Game", font=("Arial", 20, "bold"), bg='lightblue')
        self.title_label.place(x=180, y=10)

        self.level_label = tk.Label(self.root, text=f"Level {self.level}", font=("Arial", 14), bg='lightblue')
        self.level_label.place(x=270, y=50)

        self.score_label = tk.Label(self.root, text="Score: 0", font=("Arial", 14), bg='lightblue')
        self.score_label.place(x=450, y=50)

        self.create_grid()

        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit_word, font=("Arial", 14))
        self.submit_button.place(x=250, y=620)

        self.hint_button = tk.Button(self.root, text="Hint 💡", command=self.show_hint, font=("Arial", 14))
        self.hint_button.place(x=350, y=620)

        self.selected_label = tk.Label(self.root, text="", font=("Arial", 16), bg='lightblue')
        self.selected_label.place(x=200, y=580)

        self.canvas.bind("<Button-1>", self.start_drag)
        self.canvas.bind("<B1-Motion>", self.dragging)
        self.canvas.bind("<ButtonRelease-1>", self.end_drag)

    def create_grid(self):
        x_start = 150
        y_start = 100
        self.grid_buttons = []
        for i in range(GRID_SIZE):
            row = []
            for j in range(GRID_SIZE):
                btn = tk.Button(self.root, text="", width=4, height=2, font=("Arial", 14), bg="lightyellow")
                btn.place(x=x_start + j * 60, y=y_start + i * 60)
                row.append(btn)
            self.grid_buttons.append(row)

    def generate_grid_with_words(self):
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        for word in self.level_words:
            placed = False
            attempts = 0
            while not placed and attempts < 100:
                direction = random.choice(['horizontal', 'vertical'])
                if direction == 'horizontal':
                    row = random.randint(0, GRID_SIZE - 1)
                    col = random.randint(0, GRID_SIZE - len(word))
                    if all(self.grid_data[row][col + i] in ("", word[i]) for i in range(len(word))):
                        for i in range(len(word)):
                            self.grid_data[row][col + i] = word[i]
                        placed = True
                else:
                    row = random.randint(0, GRID_SIZE - len(word))
                    col = random.randint(0, GRID_SIZE - 1)
                    if all(self.grid_data[row + i][col] in ("", word[i]) for i in range(len(word))):
                        for i in range(len(word)):
                            self.grid_data[row + i][col] = word[i]
                        placed = True
                attempts += 1

        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid_data[i][j] == "":
                    self.grid_data[i][j] = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
                self.grid_buttons[i][j].config(text=self.grid_data[i][j])

    def create_letter_buttons(self):
        self.clear_letters()
        all_letters = list("".join(self.level_words))
        unique_letters = sorted(set(all_letters))
        random.shuffle(unique_letters)

        center_x = WINDOW_WIDTH // 2
        center_y = 500
        radius = 120
        angle_step = 360 / len(unique_letters)

        self.letter_coords = []
        self.letter_buttons = []

        for i, letter in enumerate(unique_letters):
            angle = math.radians(i * angle_step)
            x = center_x + radius * math.sin(angle)
            y = center_y - radius * math.cos(angle)
            self.letter_coords.append((x + 25, y + 25))  # Adjust for center
            btn = tk.Button(self.root, text=letter, font=("Arial", 16, "bold"),
                            bg="deepskyblue", fg="white", width=3,
                            command=lambda l=letter: self.select_letter(l))
            btn.place(x=x, y=y)
            self.letter_buttons.append(btn)

    def select_letter(self, letter):
        self.selected_letters.append(letter)
        current = self.selected_label.cget("text")
        self.selected_label.config(text=current + letter)

    def submit_word(self):
        word = self.selected_label.cget("text")
        if word in self.level_words and word not in self.found_words:
            self.found_words.append(word)
            self.score += 10
            self.highlight_word_in_grid(word)
            self.update_score()
            self.clear_selection()
            if all(w in self.found_words for w in self.level_words):
                messagebox.showinfo("Level Complete!", "You've found all words! Moving to next level.")
                self.level += 1
                self.setup_level()
        else:
            messagebox.showerror("Try Again", f"'{word}' is not a valid word.")
            self.clear_selection()

    def highlight_word_in_grid(self, word):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE - len(word) + 1):
                if all(self.grid_data[i][j + k] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i][j + k].config(bg="lightgreen")
                    return
        for i in range(GRID_SIZE - len(word) + 1):
            for j in range(GRID_SIZE):
                if all(self.grid_data[i + k][j] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i + k][j].config(bg="lightgreen")
                    return

    def update_score(self):
        self.score_label.config(text=f"Score: {self.score}")

    def show_hint(self):
        if self.hints_used < self.max_hints:
            remaining = [w for w in self.level_words if w not in self.found_words]
            if remaining:
                hint_word = random.choice(remaining)
                first_letter = hint_word[0]
                self.hints_used += 1
                messagebox.showinfo("Hint", f"Try a word starting with: '{first_letter}'")
            else:
                messagebox.showinfo("Hint", "You've already found all the words!")
        else:
            messagebox.showwarning("No Hints Left", "You’ve used all your hints for this level!")

    def clear_selection(self):
        self.selected_letters.clear()
        self.selected_label.config(text="")

    def clear_grid(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                self.grid_buttons[i][j].config(text="", bg="lightyellow")

    def clear_canvas(self):
        self.canvas.delete("all")

    def clear_letters(self):
        for btn in self.letter_buttons:
            btn.destroy()
        self.letter_buttons.clear()
        self.letter_coords.clear()

    def start_drag(self, event):
        self.clear_selection()
        self.track_drag(event)

    def dragging(self, event):
        self.track_drag(event)

    def track_drag(self, event):
        nearest_index = self.get_nearest_letter_index(event.x, event.y)
        if nearest_index is not None:
            letter = self.letter_buttons[nearest_index].cget("text")
            if len(self.selected_letters) == 0 or self.selected_letters[-1] != letter:
                self.select_letter(letter)

        if self.drag_line:
            self.canvas.delete(self.drag_line)

        points = [self.letter_coords[i] for i in range(len(self.selected_letters))]
        if len(points) > 1:
            self.drag_line = self.canvas.create_line(*sum(points, ()), fill="red", width=2)

    def end_drag(self, event):
        pass

    def get_nearest_letter_index(self, x, y):
        min_distance = float('inf')
        nearest_index = None
        for i, (lx, ly) in enumerate(self.letter_coords):
            distance = math.sqrt((x - lx) ** 2 + (y - ly) ** 2)
            if distance < BUTTON_RADIUS and distance < min_distance:
                min_distance = distance
                nearest_index = i
        return nearest_index


if __name__ == "__main__":
    root = tk.Tk()
    app = WordPuzzleGame(root)
    root.mainloop()


In [54]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk  # Using Pillow for image handling
import random
import math

GRID_SIZE = 5
BUTTON_RADIUS = 30
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 700
LEVEL_IMAGES = [
    "images/mountains.jpg",  # Add image paths here for each level
    "images/jungle.jpg",
    "images/beach.jpg",
    "images/space.jpg",
    "images/forest.jpg",
    "images/desert.jpg",
    "images/ocean.jpg",
    "images/city.jpg",
    "images/sky.jpg",
    "images/snow.jpg",
]

class WordPuzzleGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Word Puzzle - Interleaved Words")
        self.root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}")
        self.root.configure(bg='lightblue')

        self.canvas = tk.Canvas(root, width=WINDOW_WIDTH, height=WINDOW_HEIGHT, bg='lightblue', highlightthickness=0)
        self.canvas.pack()

        self.grid_buttons = []
        self.letter_buttons = []
        self.letter_coords = []
        self.selected_letters = []
        self.drag_line = None
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        self.score = 0
        self.level = 1
        self.hints_used = 0
        self.max_hints = 2
        self.found_words = []

        self.level_word_bank = {
            1: ["APPLE", "MANGO", "PEACH", "BERRY", "GRAPE"],
            2: ["LION", "TIGER", "ZEBRA", "BEAR", "HORSE"],
            3: ["RIVER", "OCEAN", "DESERT", "FOREST", "MOUNTAIN"],
            4: ["PYTHON", "JAVA", "CPLUS", "HTML", "CSS"],
            5: ["PLANE", "CAR", "BICYCLE", "BUS", "TRAIN"],
            6: ["ELEPHANT", "LION", "TIGER", "CROC", "BEAR"],
            7: ["EARTH", "MOON", "MARS", "VENUS", "SATURN"],
            8: ["GOLD", "SILVER", "DIAMOND", "PLATINUM", "COPPER"],
            9: ["OCEAN", "RIVER", "LAKE", "POND", "SEA"],
            10: ["COMPUTER", "LAPTOP", "MOBILE", "DESKTOP", "SERVER"]
        }

        self.create_widgets()
        self.setup_level()

    def setup_level(self):
        self.found_words = []
        self.hints_used = 0
        self.clear_grid()
        self.clear_canvas()
        self.clear_letters()
        self.level_words = self.pick_words_for_level()
        self.level_label.config(text=f"Level {self.level}")
        self.generate_grid_with_random_letters()
        self.create_letter_buttons()
        self.set_background_image()

    def pick_words_for_level(self):
        return self.level_word_bank.get(self.level, random.sample(list(self.level_word_bank[1]), 5))

    def create_widgets(self):
        self.title_label = tk.Label(self.root, text="Word Puzzle Game", font=("Arial", 20, "bold"), bg='lightblue')
        self.title_label.place(x=180, y=10)

        self.level_label = tk.Label(self.root, text=f"Level {self.level}", font=("Arial", 14), bg='lightblue')
        self.level_label.place(x=270, y=50)

        self.score_label = tk.Label(self.root, text="Score: 0", font=("Arial", 14), bg='lightblue')
        self.score_label.place(x=450, y=50)

        self.create_grid()

        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit_word, font=("Arial", 14))
        self.submit_button.place(x=250, y=620)

        self.hint_button = tk.Button(self.root, text="Hint 💡", command=self.show_hint, font=("Arial", 14))
        self.hint_button.place(x=350, y=620)

        self.selected_label = tk.Label(self.root, text="", font=("Arial", 16), bg='lightblue')
        self.selected_label.place(x=200, y=580)

        self.canvas.bind("<Button-1>", self.start_drag)
        self.canvas.bind("<B1-Motion>", self.dragging)
        self.canvas.bind("<ButtonRelease-1>", self.end_drag)

    def set_background_image(self):
        try:
            img_path = LEVEL_IMAGES[self.level - 1]  # Image path based on level
            img = Image.open(img_path)
            img = img.resize((WINDOW_WIDTH, WINDOW_HEIGHT), Image.ANTIALIAS)
            bg_image = ImageTk.PhotoImage(img)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=bg_image)
            self.canvas.image = bg_image  # Keep a reference to the image
        except Exception as e:
            print(f"Error loading image for level {self.level}: {e}")

    def create_grid(self):
        x_start = 150
        y_start = 100
        self.grid_buttons = []
        for i in range(GRID_SIZE):
            row = []
            for j in range(GRID_SIZE):
                btn = tk.Button(self.root, text="", width=4, height=2, font=("Arial", 14), bg="lightyellow")
                btn.place(x=x_start + j * 60, y=y_start + i * 60)
                row.append(btn)
            self.grid_buttons.append(row)

    def generate_grid_with_random_letters(self):
        self.grid_data = [["" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                self.grid_data[i][j] = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
                self.grid_buttons[i][j].config(text=self.grid_data[i][j])

    def create_letter_buttons(self):
        self.clear_letters()
        all_letters = list("".join(self.level_words))
        unique_letters = sorted(set(all_letters))
        random.shuffle(unique_letters)

        center_x = WINDOW_WIDTH // 2
        center_y = 500
        radius = 120
        angle_step = 360 / len(unique_letters)

        self.letter_coords = []
        self.letter_buttons = []

        for i, letter in enumerate(unique_letters):
            angle = math.radians(i * angle_step)
            x = center_x + radius * math.sin(angle)
            y = center_y - radius * math.cos(angle)
            self.letter_coords.append((x + 25, y + 25))  # Adjust for center
            btn = tk.Button(self.root, text=letter, font=("Arial", 16, "bold"),
                            bg="deepskyblue", fg="white", width=3,
                            command=lambda l=letter: self.select_letter(l))
            btn.place(x=x, y=y)
            self.letter_buttons.append(btn)

    def select_letter(self, letter):
        self.selected_letters.append(letter)
        current = self.selected_label.cget("text")
        self.selected_label.config(text=current + letter)

    def submit_word(self):
        word = self.selected_label.cget("text")
        if word in self.level_words and word not in self.found_words:
            self.found_words.append(word)
            self.score += 10
            self.highlight_word_in_grid(word)
            self.update_score()
            self.clear_selection()
            if all(w in self.found_words for w in self.level_words):
                messagebox.showinfo("Level Complete!", "You've found all words! Moving to next level.")
                self.level += 1
                if self.level > 10:
                    messagebox.showinfo("Congratulations!", "You've completed all levels!")
                    self.root.quit()
                self.setup_level()
        else:
            messagebox.showerror("Try Again", f"'{word}' is not a valid word.")
            self.clear_selection()

    def highlight_word_in_grid(self, word):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE - len(word) + 1):
                if all(self.grid_data[i][j + k] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i][j + k].config(bg="lightgreen")
                    return
        for i in range(GRID_SIZE - len(word) + 1):
            for j in range(GRID_SIZE):
                if all(self.grid_data[i + k][j] == word[k] for k in range(len(word))):
                    for k in range(len(word)):
                        self.grid_buttons[i + k][j].config(bg="lightgreen")
                    return

    def update_score(self):
        self.score_label.config(text=f"Score: {self.score}")

    def show_hint(self):
        if self.hints_used < self.max_hints:
            remaining = [w for w in self.level_words if w not in self.found_words]
            if remaining:
                hint_word = random.choice(remaining)
                first_letter = hint_word[0]
                self.hints_used += 1
                messagebox.showinfo("Hint", f"Try a word starting with: '{first_letter}'")
            else:
                messagebox.showinfo("Hint", "You've already found all the words!")
        else:
            messagebox.showwarning("No Hints Left", "You’ve used all your hints for this level!")

    def clear_selection(self):
        self.selected_letters.clear()
        self.selected_label.config(text="")

    def clear_grid(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                self.grid_buttons[i][j].config(text="", bg="lightyellow")

    def clear_canvas(self):
        self.canvas.delete("all")

    def clear_letters(self):
        for btn in self.letter_buttons:
            btn.destroy()
        self.letter_buttons.clear()
        self.letter_coords.clear()

    def start_drag(self, event):
        self.clear_selection()
        self.track_drag(event)

    def dragging(self, event):
        self.track_drag(event)

    def track_drag(self, event):
        nearest_index = self.get_nearest_letter_index(event.x, event.y)
        if nearest_index is not None:
            letter = self.letter_buttons[nearest_index].cget("text")
            if len(self.selected_letters) == 0 or self.selected_letters[-1] != letter:
                self.select_letter(letter)

    def end_drag(self, event):
        pass

    def get_nearest_letter_index(self, x, y):
        closest_distance = float('inf')
        closest_index = None
        for i, (lx, ly) in enumerate(self.letter_coords):
            distance = math.sqrt((lx - x) ** 2 + (ly - y) ** 2)
            if distance < BUTTON_RADIUS:
                if distance < closest_distance:
                    closest_distance = distance
                    closest_index = i
        return closest_index


if __name__ == "__main__":
    root = tk.Tk()
    game = WordPuzzleGame(root)
    root.mainloop()


Error loading image for level 1: [Errno 2] No such file or directory: 'images/mountains.jpg'


In [4]:
words = """
apple banana orange mango grape peach strawberry watermelon cherry lemon
dog cat horse cow lion tiger bear elephant zebra monkey kangaroo
car bus truck train plane boat helicopter bicycle scooter
table chair sofa bed cupboard drawer mirror desk shelf lamp
pen pencil notebook book eraser sharpener ruler marker paper file
math science history geography english biology chemistry physics computer art
pizza burger sandwich pasta rice noodles soup steak salad curry
teacher student principal class exam subject school college university homework
pakistan india america canada germany france china japan brazil egypt
football cricket basketball tennis volleyball baseball golf rugby badminton hockey
house apartment bungalow villa hut tent igloo cabin mansion palace
shirt jeans jacket coat sweater scarf gloves socks shoes boots
sun moon star cloud sky rainbow snow rain wind thunder
river mountain lake ocean sea desert forest jungle hill valley
phone laptop tablet monitor keyboard mouse printer speaker camera headphone
spoon fork knife plate bowl cup glass jug pan kettle
love happy sad angry scared surprised excited bored tired brave
dance sing read write draw paint play cook travel swim
gold silver copper iron steel bronze diamond ruby pearl emerald
bread butter cheese milk yogurt cream egg cereal jam honey
door window wall floor ceiling roof stairs balcony gate fence
plant tree flower grass leaf root seed fruit vegetable herb
red blue green yellow black white pink purple orange brown
"""

# Save to file
with open("words.txt", "w") as f:
    word_list = list(set(words.upper().split()))
    word_list.sort()
    for word in word_list:
        f.write(word + "\n")

print(f"{len(word_list)} words written to words.txt")


229 words written to words.txt
